<a href="https://colab.research.google.com/github/AdityaMayukhSom/hallucination-detection-pipeline/blob/main/notebooks/DatasetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup Commands

These commands needs to be run in every session of Colab. Sets up Google Drive for dataset access, logger format and generates constants used throughout the notebook.

#### Mount Google Drive

In [ ]:
from huggingface_hub import login
from google.colab import drive, userdata

drive.mount('/content/drive/')
HF_TOKEN = userdata.get('HF_TOKEN')

# enter a huggingface token, which will authenticate this notebook session
# on huggingface server, after which we can publish the dataset. An alternate
# to `login()` is `notebook_login()`. This will prompt for a token.
login(token = HF_TOKEN)

#### Install Libraries

In [ ]:
%%capture
%pip install -U loguru
%pip install -U unsloth
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U trl
%pip install -U xformers

#### Setup Imports And Logger

In [ ]:
# import logging
# log_format = "%(asctime)s - %(levelname)s - %(message)s"
# logging.basicConfig(level=logging.INFO, format=log_format, force=True)
# logger = logging.getLogger()

import gc
import os

import torch
import psutil
import ctypes
import pandas as pd
from pathlib import Path
from loguru import logger
from datasets import load_dataset, load_from_disk, Dataset
from unsloth import FastLanguageModel, is_bfloat16_supported

### Generate Smaller Dataset For Easier Processing

Loaded the original dataset from huggingface to generate a sliced smalled version which is easier to process.

If we do not mention `split` argument while in `load_dataset` function, three seperate folders will be downloaded for `train`, `test` and `validation`. In that case, the `dataset` object generated can be indexed for the three splits which will then return the dataset which contains the actual rows. Mentioning which split to load during in `load_dataset` will return the actual dataset on which we can index with the attribute names.

In [ ]:
def generate_small_dataset(
    *,
    ori_dataset_dir: str | None = None,
    dup_dataset_dir: str | None = None,
    dup_csv_file_name: str = "duplicate_mixsub.csv",
    num_rows_in_dup: int = 3000
):
    if ori_dataset_dir is not None and os.path.exists(ori_dataset_dir):
        dataset = load_from_disk(ori_dataset_dir)
    else:
        dataset = load_dataset("TRnlp/MixSub")
        if ori_dataset_dir is not None:
            dataset.save_to_disk(ori_dataset_dir)

    train_ds = dataset['train']
    test_ds = dataset['test']
    validation_ds = dataset['validation']

    # Selects num rows from the original and generates a small dataset
    # with first `num_rows_in_dup` entries.
    small_ds = train_ds.select(range(num_rows_in_dup))

    # Converts the huggingface Datasets type to Pandas Dataframe type
    small_pd_df = small_ds.to_pandas()

    if dup_dataset_dir is not None:
        if not os.path.exists(dup_dataset_dir):
            Path(dup_dataset_dir).mkdir(parents=True, exist_ok=True)
        if dup_csv_file_name is not None:
            dup_ds_path = os.path.join(dup_dataset_dir, dup_csv_file_name)
            logger.info("small csv save started")
            small_pd_df.to_csv(dup_ds_path, index=False)
            logger.success("small csv save success")

    return small_pd_df

Checks whether the path to save the small dataset exists or not, if not, creates the directory to in which it will save the generated file. It writes the small dataset in CSV format, which is different from the original Apache Arrow format.

### Generate Hallucinated Dataset

#### Create and setup unsloth model for inference

In [ ]:
def get_inference_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
        max_seq_length = 2048,
        load_in_4bit = True,
        dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
    )

    FastLanguageModel.for_inference(model)
    return model, tokenizer

#### Hallucinated Highlight Generator

##### Dataset Loader For Hallucinated Dataset

In [ ]:
def get_dataset(
    *,
    dup_dataset_dir: str | None = None,
    dup_csv_file_name: str = "duplicate_mixsub.csv",
    get_small_dataset = True
):
    if get_small_dataset and dup_dataset_dir is not None:
        # Loading only the Abstract section of the dataframe into huggingface dataset
        ds_path = Path(os.path.join(dup_dataset_dir, dup_csv_file_name))
        custom_df = pd.read_csv(ds_path, index_col = None)
        abstract_ds = Dataset.from_pandas(custom_df)
    else:
        # Loading the dataset dict from huggingface
        abstract_dsd = load_dataset("TRnlp/MixSub")
        abstract_ds = abstract_dsd['train']

    return abstract_ds

In [ ]:
def generate_hallucinated_highlights(dataset_row, model, tokenizer):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(device)

    instruction = """
    You're instructed to generate scientifically inaccurate or hallucinated highlights of the provided passage
    only without additional sentences like headings, introductions, or text before or after the generated output
    as the output will be directly used as highlight in a custom dataset. The highlight should sound plausible
    but contain incorrect information.Generate 3-5 concise highlight points from the provided research paper abstract,
    covering key contributions, methods, and outcomes. Each point should contain 10 to 15 words only. Return the
    points in plain text format without bullets.

    No Additional Commentary: Exclude lines like "Here are 3-5 concise highlight points".
    """

    init_identifier = "<|start_header_id|>assistant<|end_header_id|>"
    term_identifier = "<|eot_id|>"

    batch_data = [
        [
            {"role": "system", "content": instruction},
            {"role": "user", "content": abstract}
        ]
        for abstract in dataset_row['Abstract']
    ]

    inputs = tokenizer.apply_chat_template(
        batch_data, add_generation_prompt=True, tokenize = True,
        padding = True, truncation = True, return_tensors = 'pt',
        return_dict = True
    ).to(device)

    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, pad_token_id = tokenizer.eos_token_id)
    decoded_texts = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens = False)

    # the generaed output contains the prompt, init identifier, generated highlight and term identifier
    # the following code splits the output with init identifier, takes the second section which contains
    # the generated highlight followed by term identifier, now splits the second section based on term
    # identifier, takes the first section, which contains only the generated output. Then it strips the
    # generated content to get rid of any white spaces from the beginning and the end, and replaces
    # newline character with whitespace.

    hallucination = [decoded_text.split(term_identifier)[0].strip().replace("\n", " ") for decoded_text in decoded_texts]

    del batch_data
    del inputs
    del outputs

    # row_json = [
    #     {"role": "system", "content": instruction},
    #     {"role": "user", "content": dataset_row['Abstract']}
    # ]

    # model_prompt = tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=True)
    # model_inputs = tokenizer(model_prompt, return_tensors='pt', padding=True, truncation=True).to(device)
    # model_outputs = model.generate(**model_inputs, max_new_tokens=128, num_return_sequences=1)
    # decoded_text = tokenizer.batch_decode(model_outputs[:, model_inputs.input_ids.shape[1]:], skip_special_tokens=False)
    # hallucination = decoded_text.split(init_identifier)[1].split(term_identifier)[0].strip().replace("\n", " ")

    # del model_prompt
    # del model_inputs
    # del model_outputs
    # del decoded_text

    del instruction
    del init_identifier
    del term_identifier

    # gc.collect()
    # torch.cuda.empty_cache()

    return {
        'Hallucination': hallucination,
    }


In [ ]:
def print_mem_stats():
    stats = psutil.virtual_memory()
    free_gb = stats.free / 1e9
    print(f"Your runtime has {free_gb:.1f} gigabytes of free RAM")
    used_gb = stats.used / 1e9
    print(f"Your runtime has {used_gb:.1f} gigabytes of used RAM")
    avlb_gb = stats.available / 1e9
    print(f"Your runtime has {avlb_gb:.1f} gigabytes of available RAM")
    ram_gb = stats.total / 1e9
    print(f"Your runtime has {ram_gb:.1f} gigabytes of total RAM")
    print(f"Your runtime has {stats.percent:.1f}% usage of RAM")

In [ ]:
def generate_hallucinated_highlight_dataset():
    ori_dataset_dir = "/content/drive/MyDrive/Datasets/MixSub/Original"
    dup_dataset_dir = "/content/drive/MyDrive/Datasets/MixSub/Duplicate"

    hal_ds_hf_name = "AdityaMayukhSom/MixSub-With-Hallucinated-Highlights"
    hal_dataset_dir = "/content/drive/MyDrive/Datasets/MixSub/Hallucinated"
    hal_csv_file_name = "hallucinated_mixsub.csv"

    generate_small_dataset(
        ori_dataset_dir=ori_dataset_dir,
        dup_dataset_dir=dup_dataset_dir,
    )
    # small_pd_df.head()

    abstract_ds = get_dataset(dup_dataset_dir = dup_dataset_dir)
    model, tokenizer = get_inference_model()

    # Map the existing dataset rows into hallucinated highlights, the returned
    # dictionary from the function passed to map, will automatically be appended
    # to the dataset on which the map function is being called, and a new dataset
    # will be returned, so note that mapping does not modify the dataset on which
    # it is being called on.
    logger.info("hallucinated dataset generation started")
    hal_ds = abstract_ds.map(
        lambda row: generate_hallucinated_highlights(row, model, tokenizer),
        batched = True,
        batch_size = 4,
    )
    logger.success("hallucinated dataset generation finished")

    if not os.path.exists(hal_dataset_dir):
        Path(hal_dataset_dir).mkdir(parents=True, exist_ok=True)
    hal_ds_path = os.path.join(hal_dataset_dir, hal_csv_file_name)

    logger.info("started saving hallucinated dataset")
    hal_ds.to_csv(hal_ds_path, index=False)
    logger.success("hallucinated dataset saved to google drive as csv")

    logger.info("started pushing hallucinated dataet to huggingface")
    hal_ds.push_to_hub(hal_ds_hf_name)
    logger.success("hallucinated dataset saved to huggingface as hf dataset")

if __name__ == '__main__':
    print_mem_stats()
    libc = ctypes.CDLL("libc.so.6") # clearing cache
    libc.malloc_trim(0)
    print_mem_stats()
    generate_hallucinated_highlight_dataset()